In [2]:
import random
import time
from scapy.all import *

# Create an IP packet
ip_packet = IP(src="192.168.1.1", dst="192.168.1.2")

# Create a UDP segment
udp_segment = UDP(sport=12345, dport=80)

# Create a raw layer with some data
data = "Hello, Scapy!"
raw_layer = Raw(load=data)

# Combine the layers to create the packet
packet = ip_packet / udp_segment / raw_layer

# Number of packets to send
num_packets = 20

# Probability of packet loss (0.0 - 1.0, e.g., 0.2 for 20% loss)
packet_loss_probability = 0.2

# Maximum delay between packets (in seconds)
max_delay = 0.5

# Send the packets in a loop
for _ in range(num_packets):
    if random.random() > packet_loss_probability:
        send(packet, loop=0)  # Set loop=0 to send the packet once
        print("Packet sent")
    else:
        print("Packet lost")

    # Random delay between packets
    time.sleep(random.uniform(0, max_delay))

print(f"Sent {num_packets} packets.")


Packet lost

Sent 1 packets.
Packet sent
Packet lost

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent
Packet lost

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent
Packet lost

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent

Sent 1 packets.
Packet sent
Sent 20 packets.
